In [20]:
import csv
import collections
from math import log
import numpy as np
import pandas
import graphviz
def split_train_test(data,test_ratio):
    """
    分测试集和训练集
    
    可能会在后剪枝中用到测试集？？
    """
    np.random.seed(1)
    randomlist = np.random.permutation(len(data))
    test_set_size = round(int(len(data)) * test_ratio)#测试集的大小
    test=data[:test_set_size,:]
    train= data[test_set_size:,:]
    return test,train

data = pandas.read_csv('diabetes.csv')
data = data.values
test,rows = split_train_test(data,0.2)#rows是训练集
# print(test)
print(type(rows))

<class 'numpy.ndarray'>


In [30]:
class DecisionTree(object):
    def __init__(self,index=None,value=None,left=None,right=None,result=None):
        self.index = index       # 一个数字  列索引值
        self.value = value   # 分隔值
        self.left = left     # 左子树
        self.right = right   # 右子树
        self.result = result # 0或1
        
        
def major_class(rows):
    """
    返回最多样本所在的类
    Parameters:
    -----------
    rows:数据集
    
    Returns:
    -------    
    result:分类值：0或1
    """
    results = {}
    for row in rows:
        value = row[-1]
        if value not in results:
            results[value] = 0
        results[value] += 1
    if results[1]>=results[0]:
        result = 1
    else:
        result = 0
    return result

def entropy(rows):
    """
    计算信息熵
    Parameters:
    -----------
    rows:数据集
    
    Returns:
    -------    
    entropy:信息熵
    """
    m = len(rows)
    results = major_class(rows)
    entropy = 0
    for value in results:
        p = float(results[value])/m
        entropy -= p*log(p,2)
    return entropy
    
    
def Gini(rows):
    """
    计算基尼系数
    Parameters:
    -----------
    rows:数据集
    
    Returns:
    -------    
    Gini
    """
    m=len(rows)
    labelcount = {}
    Gini = 1
    for row in rows:
        currentlabel = row[-1]
        if currentlabel not in labelcount.keys():
            labelcount[currentlabel] = 0
        labelcount[currentlabel]+=1
    for value in labelcount:
        Gini -= (value/m)**2
    return Gini      
        
        

    


class CART(DecisionTree):
    pass

def split_data(rows,index,split):
    """
    rows:数据集
    index:列索引值
    split:在CART算法中是对应的最好分隔值
    """
    left = [row for row in rows if row[index] >= split]
    right = [row for row in rows if row[index] < split]
    return (left,right)
    
    
def buildTree(rows):
    """
    递归构建决策树
    """
    if len(rows) == 0:
        return DecisionTree()
    bestGain = 0
    #i是列数
    for index in range(0,len(rows[0])-1):# 应该  要减去1因为不要最好一列？？
        values = [row[index] for row in rows]
        for value in values:
            (set1,set2) = split_data(rows,i,j)
            p = float(len(set1))/len(rows)
            gain = Gini(rows) - p*Gini(set1) - (1-p)*Gini(set2)
            if gain>bestGain and len(set1)>0 and len(set2)>0:
                bestGain = gain
                best_index = index
                best_value = value
    if bestGain > 0:
        left = buildTree(bestSets[0]) # set1
        right = buildTree(bestSets[1]) # set2
        return DecisionTree(index=best_index,
                        value=best_value,left=left,right=right)
    else:
        return DecisionTree(results=major_class(rows))
        
        
def pre_prune(DecisionTree,minGain):
    """
    预剪枝
    """
    lb,rb = [],[]
    #遍历之后大概是一堆1和0
    for v,c in DecisionTree.left.results.items():
        lb += [[v]] *c
 
    for v,c in DecisionTree.right.results.items():
        rb += [[v]] *c
    m = len(lb+rb)
    p = float(len(lb))/m
    prune_Gain = Gini(lb+tb) - p*Gini(lb) - (1-p)*Gini(rb)
    #如果增益小于某一个设定的值，就进行剪枝
    if prune_Gain < minGain:
        tree.left,tree.right = None
        tree.results = major_class(rb+lb)

            
def predict(test,tree):
    """
    计算在测试集上的错误率
    """
    m = len(test)
    n = 0
    for row in test:
        prediction = classify(row)
        if prediction == row[-1]:
            n+=1
    ratio = (m-n)/m
    return ratio
            

def REP_prune(DecisionTree,test):
    """
    似乎不怎么正确的后剪枝
    """
    current_ratio = (test,DecisionTree)
    if DecisionTree.left.results != None and DecisionTree.right.results !=None:
        DecisionTree.left,DecisionTree.right = None
        DecisionTree.results = major_class(rb+lb)
        ratio = predict(test,DecisionTree)
        if ratio <current_ratio:#剪枝后的错误率变小??
            return REP_prune(DecisionTree,test)
        else:
            DecisionTree = buildTree(rows)
            

    
def classify(rows,DecisionTree):
    """
    进行分类
    """
    if DecisionTree.result !=None:
        return DecisionTree.results
    else:
        value = rows[tree.col]
        branch = None
        if value >= DecisionTree.value:
            branch = DecisionTree.left
        else:
            branch = DecisionTree.right
    return classify(rows,branch)
        


        

In [35]:
class ID3(DecisionTree):
    pass

    def split_data(rows,index,split):
        """
        rows:数据集
        index:列索引值
        split:在ID3算法中是对应的特征值
        """
        left = [row for row in rows if row[index] == split]
        right = [row for row in rows if row[index] !=split]
        return (left,right)
    

     
    
    def buildTree(rows):
    """
    递归构建决策树
    """
    if len(rows) == 0:
        return DecisionTree()
    bestGain = 0
    #i是列数
    for index in range(0,len(rows[0])-1):# 应该  要减去1因为不要最好一列？？
        values = [row[index] for row in rows]
        for value in values:
            (set1,set2) = split_data(rows,i,j)
            p = float(len(set1))/len(rows)
            gain = entropy(rows) - p*entropy(set1) - (1-p)*entropy(set2)
            if gain>bestGain and len(set1)>0 and len(set2)>0:
                bestGain = gain
                best_index = index
                best_value = value
    if bestGain > 0:
        left = buildTree(bestSets[0]) # set1
        right = buildTree(bestSets[1]) # set2
        return DecisionTree(index=best_index,
                        value=best_value,left=left,right=right)
    else:
        return DecisionTree(results=major_class(rows))
    
    
    def classify(rows,DecisionTree):
        """
        进行分类
        """
        if DecisionTree.result !=None:
            return DecisionTree.results
        else:
            value = rows[tree.col]
            branch = None
            if value == DecisionTree.value:
                branch = DecisionTree.left
            else:
                branch = DecisionTree.right
        return classify(rows,branch)
    

class C45(DecisionTree):
    pass

    def split_data(rows,index,split):
        """
        rows:数据集
        index:列索引值
        split:在C45算法中是对应的特征值
        """
        left = [row for row in rows if row[index] == split]
        right = [row for row in rows if row[index] !=split]
        return (left,right)



    def buildTree(rows):
    """
    递归构建决策树
    """
    if len(rows) == 0:
        return DecisionTree()
    bestGain = 0
    #i是列数
    for index in range(0,len(rows[0])-1):# 应该  要减去1因为不要最后一列？？
        values = [row[index] for row in rows]
        for value in values:
            (set1,set2) = split_data(rows,i,j)
            p = float(len(set1))/len(rows)
            gain = (entropy(rows) - p*entropy(set1) - (1-p)*entropy(set2))/entropy(rows)
            if gain>bestGain and len(set1)>0 and len(set2)>0:
                bestGain = gain
                best_index = index
                best_value = value
    if bestGain > 0:
        left = buildTree(bestSets[0]) # set1
        right = buildTree(bestSets[1]) # set2
        return DecisionTree(index=best_index,
                        value=best_value,left=left,right=right)
    else:
        return DecisionTree(results=major_class(rows))
    
    
    def classify(rows,DecisionTree):
        """
        进行分类
        """
        if DecisionTree.result !=None:
            return DecisionTree.results
        else:
            value = rows[tree.col]
            branch = None
            if value == DecisionTree.value:
                branch = DecisionTree.left
            else:
                branch = DecisionTree.right
        return classify(rows,branch)

IndentationError: expected an indented block (<ipython-input-35-cec671bdcc89>, line 17)

In [34]:


tree = buildTree(rows)

# 失败的可视化  还没有学懂graphviz

from graphviz import Digraph
dot = Digraph(comment='tree')
dot.node('o', ' ')
def view(DecisionTree):
    dot.node('o','feature%s>=%s' %(DecisionTree.col,DecisionTree.value))
    dot.node('L','feature%s>=%s' %(DecisionTree.col,DecisionTree.value))
    dot.node('R','feature%s<%s' %(DecisionTree.col,DecisionTree.value))

    dot.edges('o','L',Y)
    if DecisionTree.right != None:
        return view(DecisionTree.right)    
    dot.edges('o','R',N)
    if DecisionTree.right != None:
        return view(DecisionTree.right)
view(DecisionTree)

SyntaxError: invalid syntax (<ipython-input-34-d0d9fc186bae>, line 14)

In [11]:
#尝试实现随机森林

data = pandas.read_csv('diabetes.csv')
data = data.values
def randomTrees(n,rows):
    """
    用有放回的方式 随机生成n组数据
    
    rows:数据集
    randomTrees：n组数据
    """
    np.random.shuffle(rows)#打乱数据
    randomTrees = []
    h = rows.shape[0]
    for i in range(n):
        index = np.random.choice(h,h,replace=True)#重排列
        randomTree = rows[index,:]
        randomTrees.append(randomTree)
    return randomTrees
a=randomTrees(10,data)
print(a)


#进行投票
class_0 = 0
class_1 = 1
for i in range(len(a)-1):
    result = classify(a[i],DecisionTree)
    if result == 0:
        class_0 += 1
    else:
        class_1 +=1

if class_0 >= class_1:
    random_forest_result = 0
else:
    random_forest_result = 1
    
#我发现我随机选择了特征之后就把这个特征是啥给搞没了  所以暂时还是不用随机选择特征了  这个随机森林是一个只有随机选择数据的随机森林     
# def randomFeatures(rows):
#     """
#     随机选取特征
#     rows:数据集
    
    
#     """
    
#     n=rows.shape[1]#特征的数量
#     subset = randomTrees(n,rows)
#     m=len(subset)
#     max_feature=int(np.sqrt(n))#一颗树选取的最大特征 选择给n开方
#     index = np.random.choice(n,max_feature)
#     subset=subset[i]
#     subset_x=np.array(subset)[:,index]
#     subset_y=subset[:,-1,np.newaxis]
        
#     subset=np.hstack((subset_x,subset_y))
#     return subset

# trees = []
# for i in range(len(a)-1):
#     subset = a[i]
#     subset = randomFeatures(subset)
#     trees.append(subset)

# print(trees)



[array([[6.000e+00, 1.370e+02, 6.100e+01, ..., 1.510e-01, 5.500e+01,
        0.000e+00],
       [0.000e+00, 1.790e+02, 9.000e+01, ..., 6.860e-01, 2.300e+01,
        1.000e+00],
       [3.000e+00, 8.300e+01, 5.800e+01, ..., 3.360e-01, 2.500e+01,
        0.000e+00],
       ...,
       [2.000e+00, 1.230e+02, 4.800e+01, ..., 5.200e-01, 2.600e+01,
        0.000e+00],
       [3.000e+00, 1.730e+02, 8.200e+01, ..., 2.137e+00, 2.500e+01,
        1.000e+00],
       [4.000e+00, 1.270e+02, 8.800e+01, ..., 5.980e-01, 2.800e+01,
        0.000e+00]]), array([[  7.   , 142.   ,  60.   , ...,   0.687,  61.   ,   0.   ],
       [  1.   , 172.   ,  68.   , ...,   0.702,  28.   ,   1.   ],
       [  4.   , 132.   ,   0.   , ...,   0.302,  23.   ,   1.   ],
       ...,
       [  3.   , 142.   ,  80.   , ...,   0.2  ,  63.   ,   0.   ],
       [  1.   , 121.   ,  78.   , ...,   0.261,  28.   ,   0.   ],
       [  2.   ,  94.   ,  76.   , ...,   0.649,  23.   ,   0.   ]]), array([[1.00e+01, 7.50e+01, 8.20e+0

AttributeError: type object 'DecisionTree' has no attribute 'result'

IndentationError: expected an indented block (<ipython-input-17-200afdd3eb93>, line 3)

In [58]:
#一些无关的测试

x=np.array([1,2,3,4,5,6,7.8])
m=x.shape[0]
print(m)
x=x.reshape(7,1)
y=np.array([213,4324,45353,456456,456456,45,6])
y = y.reshape(m,1)
print(y)
x_y = np.hstack((x,y))

print(x_y)
np.random.shuffle(x_y)
print(x_y)
dataset=[]


7
[[   213]
 [  4324]
 [ 45353]
 [456456]
 [456456]
 [    45]
 [     6]]
[[1.00000e+00 2.13000e+02]
 [2.00000e+00 4.32400e+03]
 [3.00000e+00 4.53530e+04]
 [4.00000e+00 4.56456e+05]
 [5.00000e+00 4.56456e+05]
 [6.00000e+00 4.50000e+01]
 [7.80000e+00 6.00000e+00]]
[[2.00000e+00 4.32400e+03]
 [3.00000e+00 4.53530e+04]
 [4.00000e+00 4.56456e+05]
 [1.00000e+00 2.13000e+02]
 [7.80000e+00 6.00000e+00]
 [5.00000e+00 4.56456e+05]
 [6.00000e+00 4.50000e+01]]


In [59]:
x_y=np.array([[11,321,342],
            [23,42,423],
            [435,324,243]])

m=3
index = np.random.choice(m,m,replace=True)
print(index)
bootstrap_x_y = x_y[index,:]
print(bootstrap_x_y)

[2 0 0]
[[435 324 243]
 [ 11 321 342]
 [ 11 321 342]]
